In [ ]:
pip install pyLDAvis

In [ ]:
pip install langdetect

     |████████████████████████████████| 981 kB 9.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=7fc08c0f5811d039e06d468f424081d3e48e06fcef144f11c741879044c78e08
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [1]:
import re
import nltk
import functools
import operator
import pandas as pd
import gensim
import pyLDAvis.gensim_models

from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from textblob import Word

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim.corpora import Dictionary

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [15]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
STOPWORDS = stopwords.words('english')

In [3]:
file_path = "/content/drive/MyDrive/Hackathon/NanoGiants3/datasets/DisneylandReviews.csv"

In [4]:
df = pd.read_csv(file_path)
df.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Review_ID          1008 non-null   int64 
 1   Rating             1008 non-null   int64 
 2   Year_Month         1008 non-null   object
 3   Reviewer_Location  1008 non-null   object
 4   Review_Text        1008 non-null   object
 5   Branch             1008 non-null   object
dtypes: int64(2), object(4)
memory usage: 47.4+ KB


# Preprocessing

In [6]:
def get_language_code(text):
    try:
        return detect(text)
    except LangDetectException:
        return np.nan

In [7]:
# Bottleneck on large datasets!
df["language_code"] = df.apply(lambda row: get_language_code(row["Review_Text"]), axis=1)

In [8]:
# Get none english row count
df.shape[0] - df[df["language_code"] == "en"].shape[0]

0

In [9]:
def clean_text(text: str):
    # remove and replace all urls
    text = re.sub(r'http\S+', ' ', text)

    # remove and replace none alphanumerical letters
    text = re.sub(r'\W+', ' ', text.lower())

    words = []
    for word in text.split():
        if word in STOPWORDS:
            continue
        words.append(Word(word).lemmatize())
    return " ".join(words)

In [10]:
df['text_cleaned'] = df['Review_Text'].apply(clean_text)

In [11]:
df['tokens'] = df['text_cleaned'].apply(word_tokenize)

In [12]:
df.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,language_code,text_cleaned,tokens
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong,en,ever disneyland anywhere find disneyland hong ...,"[ever, disneyland, anywhere, find, disneyland,..."
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong,en,since last time visit hk disneyland yet time s...,"[since, last, time, visit, hk, disneyland, yet..."
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong,en,thanks god hot humid visiting park otherwise w...,"[thanks, god, hot, humid, visiting, park, othe..."
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong,en,hk disneyland great compact park unfortunately...,"[hk, disneyland, great, compact, park, unfortu..."
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong,en,location city took around 1 hour kowlon kid li...,"[location, city, took, around, 1, hour, kowlon..."


# LDA

In [13]:
id2word = Dictionary(df['tokens'])
texts = df['tokens']
corpus = [id2word.doc2bow(doc) for doc in df['tokens']]

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(
    id2word=id2word,
    num_topics = 4,
    random_state=42,
    passes = 10,
    update_every=1,
    alpha='auto',
    per_word_topics=True
)

In [18]:
pyLDAvis.enable_notebook()

In [19]:
vis = pyLDAvis.gensim_models.prepare(
    lda_model,
    corpus,
    id2word
)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.000257 -0.000754       1        1  36.655439
3      0.001005  0.000074       2        1  30.905919
1     -0.000389  0.000799       3        1  18.641129
2     -0.000359 -0.000119       4        1  13.797512, topic_info=            Term       Freq      Total Category  logprob  loglift
4058     surfing   9.000000   9.000000  Default  30.0000  30.0000
960       saloon  10.000000  10.000000  Default  29.0000  29.0000
2347        mini   9.000000   9.000000  Default  28.0000  28.0000
1116         8am  10.000000  10.000000  Default  27.0000  27.0000
3455        team   9.000000   9.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
4745       acted   1.640608  10.256589   Topic4  -8.2809   0.1478
4635   essential   1.628471   9.989446   Topic4  -8.2883   0.1668
2821         lit   1.644489  11.223516   Topic4  -8.2785   0.0601
4295  marginally   1.628589  10.524228   Topic4  -8.2882   0.1147
3194   behaviour   1.629300  10.837948   Topic4  -8.2878   0.0858

[225 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2543      1  0.325645        10min
2543      2  0.325645        10min
2543      3  0.217096        10min
2543      4  0.217096        10min
3374      1  0.406583          110
...     ...       ...          ...
2750      4  0.093114  wonderfully
3492      1  0.319940        yikes
3492      2  0.319940        yikes
3492      3  0.213293        yikes
3492      4  0.213293        yikes

[812 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])